In [210]:
import numpy as np
import pandas as pd
import json
import random
import uuid
from pathlib import Path
from datetime import datetime

def calculate_balance(df):
    df['Balance'] = df['Balance'].astype(str)
    no_balance_idx = df[df['Balance'].str.strip() == ''].index
    reverse_index = no_balance_idx.tolist()[::-1]

    for idx in reverse_index:
        next_idx = idx + 1
        next_idx_balance = float(df['Balance'].iloc[next_idx])
        current_amount = df['Amount'][idx]
        current_balance = current_amount + next_idx_balance
        df.loc[idx, 'Balance'] = current_balance

    return df

def generate_transaction_id(df):
    df['Transaction_ID'] = None

    for idx in range(df.index.stop):
        df.loc[idx,'Transaction_ID'] = str(uuid.uuid4())

    return df

def generate_missing_transaction_id(df):
    list_of_idx = df[df['Transaction_ID'].isnull()].index
    for idx in list_of_idx:
        df.loc[idx,'Transaction_ID'] = str(uuid.uuid4())
    return df

# Define data folder
data_folder = Path.cwd().parent / 'data'
transactions_folder = data_folder / 'transactions'

# Load in Chase transactions
chase_df = pd.read_csv(transactions_folder / "chase_checkings_transactions_master.csv", index_col=False)
chase_savings = pd.read_csv(transactions_folder / "chase_savings_transactions_master.csv", index_col=False)
new_chase_df = pd.read_csv(transactions_folder / "chase_checkings_transactions_new.csv", index_col=False)
new_chase_savings = pd.read_csv(transactions_folder / "chase_savings_transactions_new.csv", index_col=False)

# Load AMEX transactions
amex_df = pd.read_csv(transactions_folder / "amex_transactions_master.csv")
new_amex_df = pd.read_csv(transactions_folder / "amex_transactions_new.csv")

In [219]:
new_chase_savings['Balance'] = new_chase_savings['Balance'].astype(str)

In [220]:
# Update balances
chase_df = calculate_balance(chase_df)
chase_savings = calculate_balance(chase_savings)

# Add in transaction ids
chase_df = generate_transaction_id(chase_df)

# Add new transactions to chase master
chase_df = chase_df.merge(
    right=new_chase_df,
    how='right')

chase_savings = chase_savings.merge(
    right=new_chase_savings,
    how='right'
)
# Add new amex transactions
amex_df = amex_df.merge(
    right=new_amex_df,
    how='right'
)

# Add transaction ids for new transactions
chase_df = generate_missing_transaction_id(chase_df)
chase_savings = generate_missing_transaction_id(chase_savings)
amex_df = generate_missing_transaction_id(amex_df)

In [ ]:
# Save the master dataframes with transaction_ids for later
chase_df.to_csv(transactions_folder / "chase_checkings_transactions_master.csv", index = False)
chase_savings.to_csv(transactions_folder / "chase_savings_transactions_master.csv", index = False)
amex_df.to_csv(transactions_folder / "amex_transactions_master.csv", index = False)

In [221]:
# Rename columns for chase
chase_df = chase_df.rename(columns={"Posting Date":"Date"})

# Get the current checking and saving balance
no_balance_transactions = chase_df[chase_df['Balance'].str.strip() == '']
process_amount = no_balance_transactions['Amount'].sum()
most_recent_balance = float(chase_df[~(chase_df['Balance'].str.strip() == '')].iloc[0].Balance)
checking_balance = round(most_recent_balance + process_amount,2)
savings_balance = float(chase_savings.iloc[0].Balance)

# Keep only relevant columns in Chase
chase_df = chase_df[["Date", "Description", "Amount", "Transaction_ID"]]

# Convert date column to proper format
chase_df['Date'] = pd.to_datetime(chase_df['Date'])
amex_df['Date'] = pd.to_datetime(amex_df['Date'])

# Add bank identifier
chase_df['Bank'] = 'Chase'
amex_df['Bank'] = 'AMEX'

# Multiply amount by -1 so debits are negative
amex_df['Amount'] = amex_df['Amount'] * -1

# Combine both DataFrames
transactions_df = pd.concat([chase_df,amex_df])

# Save merged data to csv file
transactions_df.to_csv(transactions_folder / "merged_transactions.csv", index = False)

# Save balances to a JSON file
balances = {
    "chase_checkings_balance": checking_balance,
    "chase_savings_balance": savings_balance 
}

with open(data_folder / "reports/chase_balances.json", "w") as file:
    json.dump(balances, file)